# Lab3: Diabetes Prediction with BigQuery AutoML  
### Supporting Diabetes research:  

BigQuery ML enables users to create and execute machine learning models in BigQuery using SQL queries. The goal is to democratize machine learning by enabling SQL practitioners to build models using their existing tools and to increase development speed by eliminating the need for data movement.
In this tutorial, you use the sample Covid19 dataset for BigQuery.
## Objectives
In this tutorial, you will use BigQuery to explore immunological data, AutoML to automatically generate ML model for peptide binding. Also, you will leverage BQML to explore vareous ML models and perform feature engineering:
+ BigQuery `ML.CREATE` to create a classification model using the `CREATE MODEL` statement
+ The `ML.EVALUATE` function to evaluate the ML model
+ Use `ML.TRANSFORM`feature engineering functions to improve model performance
+ The `ML.PREDICT` function to make predictions using the ML model

# AutoML to explore classification model
Now as we explored Classification model, we might be interested to explore what other models can we used to make better prediction. 
GCP provide a service AutoML 
which allow scientist to submit their data and 
to explore vareous ML models.
+ [AutoML models](https://pantheon.corp.google.com/automl-tables/locations/us-central1/datasets/TBL8658818995480166400;modelId=TBL2508683113329065984;task=basic/schemav2?project=covid-19-271622) can be explored through GCP console
+ You can also call [AutoML from BQML](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-automl)


In [1]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GCP_PROJECT_ID=shell_output[0]
print("GCP project ID:" + GCP_PROJECT_ID)

GCP project ID:covid-19-271622



# Leverage AutoML to build the model

Building ML models with BigQuery AutoML is as simple as writing SQL statements; makes ML modeling accessible to even SQL developers and analysts. We will create a model to predict for a given peptide if there is strong binding affinity with certain HLA Allele.

Following statement creates a classification model using logistic regression. We are selecting feature columns of Allele and peptide of specific mers to classify if a peptide is a good candidate for vaccine testing.

    Note: we are filtering data for peptides with length of 9 or 10 mers only. Also, since we run multiple samples, we are randomizing samples by 80% of data for learning.



In [3]:
%%bigquery --project $GCP_PROJECT_ID
CREATE OR REPLACE MODEL `diabetes.Classification_model_automl`
OPTIONS
(
model_type='automl_classifier',
input_label_cols=['Outcome'],
budget_hours=1.0
)
AS
SELECT
 Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, Outcome
 FROM
  `covid-19-271622.diabetes.diabetes2`

""


### It will take an hour to run


# Evaluate your model

After creating your model, you evaluate the performance of the classifier using the ML.EVALUATE function. You can also use the ML.ROC_CURVE function for logistic regression specific metrics.

A classifier is one of a set of enumerated target values for a label. For example, in this tutorial you are using a classification model that detects one of the qualification class for peptide binding.

To run the ML.EVALUATE query that evaluates the model:


In [7]:
%%bigquery --project $GCP_PROJECT_ID
SELECT
  *
FROM ML.EVALUATE(MODEL `diabetes.Classification_model_automl`)

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.740741,0.666667,0.790123,0.701754,0.592812,0.793464


In [8]:
%%bigquery --project $GCP_PROJECT_ID
SELECT roc_auc,
       CASE WHEN roc_auc > .8 THEN 'good'
            WHEN roc_auc > .7 THEN 'fair'
            WHEN roc_auc > .5 THEN 'not great'
            ELSE 'poor' END AS model_quality
FROM ML.EVALUATE(MODEL `corona.Classification_model_automl`)

,roc_auc,model_quality
0,0.982554,good


# You can check vareous models explored by AutoML 
+ AutoML keep [log](https://pantheon.corp.google.com/automl-tables/locations/us-central1/datasets/TBL8658818995480166400;modelId=TBL2508683113329065984;task=basic/train?project=covid-19-271622) of all model configurations it explored
+ [Models](https://pantheon.corp.google.com/logs/query;query=resource.type%3D%22cloudml_job%22%20resource.labels.job_id%3D%22TBL2508683113329065984%22%20resource.labels.project_id%3D%22covid-19-271622%22%20labels.log_type%3D%22automl_tables%22%20jsonPayload.%22@type%22%3D%22type.googleapis.com%2Fgoogle.cloud.automl.master.TablesModelStructure%22?project=covid-19-271622)
+ [Trials](https://pantheon.corp.google.com/logs/query;query=resource.type%3D%22cloudml_job%22%20resource.labels.job_id%3D%22TBL2508683113329065984%22%20resource.labels.project_id%3D%22covid-19-271622%22%20labels.log_type%3D%22automl_tables%22%20jsonPayload.%22@type%22%3D%22type.googleapis.com%2Fgoogle.cloud.automl.master.TuningTrial%22?project=covid-19-271622)


# Run Prediction on BQML Model

Now that you have evaluated your model, the next step is to use it to predict outcomes.

To run the query that uses the model to predict the number of transactions: Following example demonstrate leveraging BQ model for prediction. Optionally, you can export model and publish it on to Google AI Platform for serving prediction.


In [9]:
%%bigquery --project $GCP_PROJECT_ID
SELECT
  *
FROM ML.PREDICT(MODEL `diabetes.Classification_model_automl`, (
  SELECT
 Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, Outcome
 FROM
  `covid-19-271622.diabetes.diabetes2`))

,predicted_Outcome,predicted_Outcome_probs,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,"[{'label': 0, 'prob': 0.8825135827064514}, {'l...",1,0,68,35,0,32.0,0.389,22,0
1,0,"[{'label': 0, 'prob': 0.8825135827064514}, {'l...",1,0,68,35,0,32.0,0.389,22,0
2,0,"[{'label': 0, 'prob': 0.9310183525085449}, {'l...",1,0,74,20,23,27.7,0.299,21,0
3,0,"[{'label': 0, 'prob': 0.9310183525085449}, {'l...",1,0,74,20,23,27.7,0.299,21,0
4,0,"[{'label': 0, 'prob': 0.595362663269043}, {'la...",5,0,80,32,0,41.0,0.346,37,1
...,...,...,...,...,...,...,...,...,...,...,...
1531,1,"[{'label': 0, 'prob': 0.21346446871757507}, {'...",2,197,70,45,543,30.5,0.158,53,1
1532,1,"[{'label': 0, 'prob': 0.1464855968952179}, {'l...",0,198,66,32,274,41.3,0.502,28,1
1533,1,"[{'label': 0, 'prob': 0.1464855968952179}, {'l...",0,198,66,32,274,41.3,0.502,28,1
1534,1,"[{'label': 0, 'prob': 0.16226908564567566}, {'...",1,199,76,43,0,42.9,1.394,22,1


The result shows predicted quality class with confidence. You can compare that with original result. Next step is to operationalize ML pipeline so that you can efficiently perform data updates and model updates. Check out AI Pipeline example for peptide prediction to learn more!

### This is end of Lab3! Congratualtions!